# Attribution methods

In this notebook I perform attribution methods with the Wts and check for correlation with the sign distributions.

In [1]:
# imports

from cnn_architecture import CNN2Model
from utils import *
from load_datasets import load_and_prep_dataset

import tensorflow_datasets as tfds
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import random

2024-06-10 11:14:51.954934: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/net/projects/scratch/summer/valid_until_31_January_2025/epetersen/miniconda3/envs/thesis_no_gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CNN2Model_adapted(tf.keras.Model):
    
    # basic
    def __init__(self):
        super(CNN2Model_adapted, self).__init__()
        
        # set biases to a value that is not exactly 0.0, so they don't get handled like pruned values
        self.bias_in = tf.keras.initializers.Constant(value=0.0000000001)
        
        self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3,activation="relu", padding="same",kernel_initializer='glorot_uniform', bias_initializer=self.bias_in) # [batchsize,32,32,64]
        self.conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3,activation="relu", padding="same",kernel_initializer='glorot_uniform', bias_initializer=self.bias_in) # [batchsize,32,32,64]
        self.maxpool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2),input_shape=(32, 32, 64)) # [batchsize,16,16,64]
        self.flatten = tf.keras.layers.Flatten() # [batch_size,16384]
        self.dense1 = tf.keras.layers.Dense(256, activation="relu",kernel_initializer='glorot_uniform', bias_initializer=self.bias_in) # [batch_size,256]
        self.dense2 = tf.keras.layers.Dense(256, activation="relu",kernel_initializer='glorot_uniform', bias_initializer=self.bias_in) # [batch_size,256]
        self.dense3 = tf.keras.layers.Dense(10, activation="softmax",kernel_initializer='glorot_uniform', bias_initializer=self.bias_in) # [batch_size,256]

    #@tf.function
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.maxpool(x)
        dense1_input = self.flatten(x)
        dense1_out = self.dense1(dense1_input)
        dense2_out = self.dense2(dense1_out)
        dense3_out = self.dense3(dense2_out)
        return dense3_out, dense1_out[0], dense2_out[0]

In [3]:
# get contribution values for neurons
def contribution_scores(test, model):

    # hyperparameters
    loss_function= tf.keras.losses.CategoricalCrossentropy()
    
    # initializeing contribution scores
    c_score_1 = np.zeros(256)
    c_score_2 = np.zeros(256)

    # iterate through testing set and get gradients for each weight
    first_round = True
    for x, t in tqdm(test):

        # get gradients 
        with tf.GradientTape() as tape:
            pred, dense1_out, dense2_out = model(x)
            loss = loss_function(t, pred)
        gradients = tape.gradient(loss, model.trainable_variables)

        # set gradients to zero for all pruned weights
        new_gradients = []
        for gradient_matrix, variables in zip(gradients, model.trainable_variables):
            gradient_matrix = tf.where(variables == 0.0, 0.0, gradient_matrix)
            new_gradients.append(gradient_matrix)
        gradients_dense2 = new_gradients[6]
        gradients_dense3 = new_gradients[8]

        # compute c score for each neuron
        for i in range(256):
            c_score_1[i] = c_score_1[i] + np.sum(np.abs(gradients_dense2[i]*dense1_out[i]))
            c_score_2[i] = c_score_2[i] + np.sum(np.abs(gradients_dense3[i]*dense2_out[i]))

    # normalize c_scores to values between 0 and 1
    c_score_1 = c_score_1 / np.sum(c_score_1)
    c_score_2 = c_score_2 / np.sum(c_score_2)
    
    return  c_score_1,c_score_2

In [4]:
def try_computing_c_scores():

    # make a model to load the weights into
    train_dataset, test_dataset = load_and_prep_dataset("CIFAR", batch_size=1, shuffle_size=512)
    model = CNN2Model_adapted()
    model(list(train_dataset)[0][0])
    weights_wt = model.get_weights()
    #print(weights_wt[0][0][0][0])
    
    # get WT weights
    #model.load_weights(f"1b WTs/old_format/WT_CIFAR_IMP_0") # does not work for thesis env, works for no_gpu, does not work for gpu
    #model.load_weights(f"1b WTs/new_format/WT_CIFAR_IMP_0.weights.h5") # does not work for thesis env, works for no_gpu, does not work for gpu
    model.load_weights(f"1b WTs/h5_format/WT_CIFAR_IMP_0.h5") # does work for thesis env, works for no_gpu, works for gpu     
    weights_wt = model.get_weights()
    #print(weights_wt[0][0][0][0])

    # plot distirbution of contribution values
    for c_score in contribution_scores(test_dataset, model):
        #print(c_score)
        plt.figure()
        plt.hist(c_score)
        plt.show()

#try_computing_c_scores()

In [5]:
# store contribution values for each sign distribution in files
def get_and_store_c_scores(datasets, n_wts):
    train_dataset, test_dataset = load_and_prep_dataset("CIFAR", batch_size=2, shuffle_size=512)
    model = CNN2Model_adapted()
    model(list(train_dataset)[0][0])

    for dataset in datasets:
        for i in range(n_wts):
            train_dataset, test_dataset = load_and_prep_dataset(dataset, batch_size=1, shuffle_size=512)
            model.load_weights(f"1b WTs/h5_format/WT_{dataset}_IMP_{i}.h5")
            c_scores_1, c_scores_2 = contribution_scores(test_dataset, model)
            np.save(f"5d Contribution values/c_scores_dense1_{dataset}_wt{i}", np.array(c_scores_1))
            np.save(f"5d Contribution values/c_scores_dense2_{dataset}_wt{i}", np.array(c_scores_2))

In [6]:
#get_and_store_c_scores(["CIFAR","CINIC","SVHN"],15)
get_and_store_c_scores(["CIFAR"],1)

100%|██████████| 10000/10000 [48:29<00:00,  3.44it/s]


In [7]:
# plot contribution values in scatterplot matrix as 5th dimension

In [8]:
# collect statistics for contribution value of each cluster